<a href="https://colab.research.google.com/github/jsebastianquiroga/DEEP_LEARNING_PUJ/blob/main/taller_dos/traffic_sign_classification_and_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">



<h1> <strong>Aprendizaje Profundo.</strong></h1> 
<h2> Para: Ing. Julio Omar Palacio Niño, M.Sc.</br></h2>
<h2>Integrantes:</br></h2>
<h2>Antonio Jose Caicedo.</br></h2>
<h2>Leonardo García.</br></h2>
<h2>Juan Sebastián Quiroga Bernal. </h2>

**Este colab puede ser ejecutado por cualquier usuario sin necesidad de cargar los archivos al entorno, debido a que la conexión está configurada para acceder a los archivos desde la nube**.


<h1> <strong>Taller 2: Traffic Sign Classification and Recognition.</strong></h1> 

En el presente Notebook, se exploraran los datos del data set de Kaggel: https://www.kaggle.com/datasets/wjybuqi/traffic-sign-classification-and-recognition

Este conjunto de datos contiene 6358 etiquetas de categorías etiquetadas manualmente. Las etiquetas incluyen las 10 categorías siguientes: `GuideSign`, `M1`, `M4`, `M5`, `M6`, `M7`, `P1`, `P10_50`, `P12`, `W1`, correspondientes a diez categorías diferentes de señales de tráfico. Todos los datos se han dividido manualmente en conjunto de entrenamiento y conjunto de prueba según la proporción.

El presente notebook, estará dividido en divido en las siguientes partes:
<ol>
<li> TBD:. </li>
<li> TBD. </li>
<li> TBD.</li>
</ol>

***

In [ ]:
# !pip install tensorflow-gpu

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f'Number of GPUs available: {len(gpus)}')
    for gpu in gpus:
        print(f'GPU detected: {gpu}')
else:
    print('No GPU detected')

## <h1><strong> 1. Cargar el data set: </h1></strong>

El dataset del link en referencia, fue descargado previamente y subido a google drive, para poder cargar el presente notebook sin problemas al momento de cargar la información. </br>

In [ ]:
# Conexion a gdrive
# from pydrive.auth import GoogleAuth
# from google.colab import drive
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# twitter = '1gwF2scW3dpklHez9kNws1tlnSrK0XxWU' 


# download = drive.CreateFile({'id': twitter}) 
# download.GetContentFile('gender-classifier-DFE-791531.csv')

In [ ]:
# Set the paths to the training and testing dataset folders
path_to_folder1 = 'train_dataset/train'
path_to_folder2 = 'test_dataset/test'

# Function to load image paths and labels from a folder
def load_image_paths_and_labels(folder_path):
    image_paths = []  # List to store image paths
    labels = []       # List to store corresponding labels

    # Iterate through each class folder in the given folder_path
    for class_index, class_name in enumerate(os.listdir(folder_path)):
        # Get all JPEG image file paths in the current class folder
        class_image_paths = glob.glob(os.path.join(folder_path, class_name, '*.jpg'))
        # Create a list of class indices, one for each image in the current class folder
        class_labels = [class_index] * len(class_image_paths)
        
        # Extend the image_paths and labels lists with the current class's data
        image_paths.extend(class_image_paths)
        labels.extend(class_labels)

    return image_paths, labels

# Call the function for both training and testing dataset folders
folder1_image_paths, folder1_labels = load_image_paths_and_labels(path_to_folder1)
folder2_image_paths, folder2_labels = load_image_paths_and_labels(path_to_folder2)

# Combine the image paths and labels from both folders into two larger lists
all_image_paths = folder1_image_paths + folder2_image_paths
all_labels = folder1_labels + folder2_labels

# Print the total number of image paths and labels in the combined lists
print("Number of image paths:", len(all_image_paths))
print("Number of labels:", len(all_labels))

In [ ]:
# Split the dataset into training and testing sets
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(all_image_paths, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

In [ ]:
# Define a function to load and preprocess a single image
def load_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size, color_mode="rgb")
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    return img_array

In [ ]:
# Set the target image size (in pixels) and batch size for data processing
pixel = 224
batch_size = 200
num_classes = len(np.unique(all_labels))

In [ ]:
# Create an ImageDataGenerator instance for the training set with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
# Create an ImageDataGenerator instance for the testing set without data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create instances of the custom data generator for the training and testing sets
train_datagen_instance = train_datagen.flow_from_dataframe(dataframe=pd.DataFrame({'image_path': train_image_paths, 'label': [str(label) for label in train_labels]}), directory=None, x_col='image_path', y_col='label', target_size=(pixel, pixel), class_mode='categorical', batch_size=batch_size)
test_datagen_instance = test_datagen.flow_from_dataframe(dataframe=pd.DataFrame({'image_path': test_image_paths, 'label': [str(label) for label in test_labels]}), directory=None, x_col='image_path', y_col='label', target_size=(pixel, pixel), class_mode='categorical', batch_size=batch_size)

In [ ]:
def plot_images(image_paths, labels, title):
    indices = np.random.choice(range(len(image_paths)), 10)
    selected_image_paths = [image_paths[i] for i in indices]
    selected_labels = [labels[i] for i in indices]
    
    images = np.array([img_to_array(load_img(img_path, target_size=(pixel, pixel))) for img_path in selected_image_paths])
    images = images / 255.0
    
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle(title, fontsize=16)

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i])
        ax.set_title(selected_labels[i])
        ax.axis('off')

    plt.show()

plot_images(train_image_paths, train_labels, 'Train Images')
plot_images(test_image_paths, test_labels, 'Test Images')

In [ ]:
# Define the model
img_height = pixel
img_width = pixel
num_channels = 3

model = Sequential([
    Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same', input_shape=(img_height, img_width, num_channels)),
    MaxPooling2D(pool_size=(5, 5)),
    Flatten(),
    Dense(units=100, activation='relu'),
    Dense(units=10, activation='softmax')
])

# Summary
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training parameters
epochs = 20

# Train the model
history = model.fit(
    train_datagen_instance,
    epochs=epochs,
    steps_per_epoch=len(train_image_paths) // batch_size
)

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_datagen_instance, steps=test_datagen_instance.n // batch_size)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_acc:.4f}')